#### World Bank Projects Dataset

In this notebook, I'll find the 10 countries with the most projects, find the top 10 major project themes, and clean the column associated with the names of the major themes.


In [104]:
import pandas as pd
import json
from pandas.io.json import json_normalize

# open json
with open('data/world_bank_projects.json') as f:
    raw = json.load(f)
  
# use normalization to create a table from the nested column
df_themes = json_normalize(raw, 'mjtheme_namecode', ['id'])
df_themes.head()

,code,name,id
0,8,Human development,P129828
1,11,,P129828
2,1,Economic management,P144674
3,6,Social protection and risk management,P144674
4,5,Trade and integration,P145310


In [105]:
# show counts for each theme - notice that 122 have no name
df_themes.name.value_counts().head(10)

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64

In [106]:
# group by code to make a mapping from name to code
df_themes_name_to_code = df_themes.groupby('name').code.max().sort_values(ascending=False)
df_themes_name_to_code

name
Urban development                                9
                                                 9
Human development                                8
Social dev/gender/inclusion                      7
Social protection and risk management            6
Trade and integration                            5
Financial and private sector development         4
Rule of law                                      3
Public sector governance                         2
Environment and natural resources management    11
Rural development                               10
Economic management                              1
Name: code, dtype: object

In [107]:
# remove row that incorrectly maps "" to code number 9 and convert it to a dataframe
df_themes_name_to_code = df_themes_name_to_code[df_themes_name_to_code.index != ""]
df_themes_name_to_code = pd.DataFrame(df_themes_name_to_code, columns = ['code'])
df_themes_name_to_code

,code
name,
Urban development,9
Human development,8
Social dev/gender/inclusion,7
Social protection and risk management,6
Trade and integration,5
Financial and private sector development,4
Rule of law,3
Public sector governance,2
Environment and natural resources management,11


In [108]:
# change the index so that it maps code to name
df_themes_name_to_code['name_clean']= df_themes_name_to_code.index
df_theme_code_to_name = df_themes_name_to_code.set_index(['code'])
df_theme_code_to_name

,name_clean
code,
9,Urban development
8,Human development
7,Social dev/gender/inclusion
6,Social protection and risk management
5,Trade and integration
4,Financial and private sector development
3,Rule of law
2,Public sector governance
11,Environment and natural resources management


In [109]:
# merge df_themes with the mapping dataframe 
df_themes_clean = df_themes.merge(df_theme_code_to_name, how = 'outer', left_on = ['code'], right_index = True)
#assign the clean names to the name column and drop the extra column
df_themes_clean['name'] = df_themes_clean['name_clean']
df_themes_clean.drop(['name_clean'], axis = 1)

,code,name,id
0,8,Human development,P129828
17,8,Human development,P127033
41,8,Human development,P132616
43,8,Human development,P146271
44,8,Human development,P146271
47,8,Human development,P146271
57,8,Human development,P146109
58,8,Human development,P146109
69,8,Human development,P143025
79,8,Human development,P122700


In [95]:
# redo the counts, and we can see there are no more missing names
df_themes_clean.name.value_counts().head(10)

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64